<a href="https://colab.research.google.com/github/sqpoonz/esomatec/blob/main/Modelmaker_esomatec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


##Step 1
### Installieren der Packeges
Model Maker packages + pycocotools library wird installiert (wird für die Auswertung benötigt) 

 [GitHub repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) 

Beim Aufbau vom Modelmaker, sowie auch beim importieren kommen Fehler die ignoriert werden können.

In [ ]:
!pip install -q tflite-model-maker
!pip install -q tflite-support

     |████████████████████████████████| 621 kB 5.2 MB/s 
     |████████████████████████████████| 1.1 MB 30.2 MB/s 
     |████████████████████████████████| 840 kB 19.7 MB/s 
     |████████████████████████████████| 87 kB 2.9 MB/s 
     |████████████████████████████████| 3.4 MB 34.3 MB/s 
     |████████████████████████████████| 1.2 MB 39.1 MB/s 
     |████████████████████████████████| 596 kB 45.1 MB/s 
     |████████████████████████████████| 6.3 MB 27.4 MB/s 
     |████████████████████████████████| 213 kB 44.1 MB/s 
     |████████████████████████████████| 1.1 MB 37.4 MB/s 
     |████████████████████████████████| 120 kB 54.9 MB/s 
     |████████████████████████████████| 25.3 MB 88 kB/s 
     |████████████████████████████████| 352 kB 49.0 MB/s 
     |████████████████████████████████| 1.1 MB 36.8 MB/s 
     |████████████████████████████████| 47.6 MB 37 kB/s 
     |████████████████████████████████| 99 kB 7.0 MB/s 
     |████████████████████████████████| 208 kB 53.2 MB/s 


In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

## Step 2
### Hochladen der Zip Datei (Bilder+XML)

In [ ]:
!mkdir /content/images
!unzip -q images.zip -d /content/images/all
!mkdir /content/images/train; mkdir /content/images/validation

###Holt sich die split.py Datei womit die Bilder+XML in die Ordner gepackt werden.

 (glaube hab es auf 90/10 aufgeteilt. kann angepasst werden 15-20% sollten aber für Validation höchstens genommen werden.


In [ ]:
!wget https://raw.githubusercontent.com/sqpoonz/esomatec/main/split.py
!python split.py

## Step 3

### Step 1: Laden der Bilder in train_data/val_data

* Bilder in  `train_data` werden benutzt um das Detection Model zu trainieren.
* Bilder in `val_data` wird benutzt um das Model zu testen, wie sich das Model zu neuen nicht gesehenen Bildern macht.

In [ ]:
train_data = object_detector.DataLoader.from_pascal_voc(
    'images/train',
    'images/train',
    ['0', '1','2','3','4','5','6','7','8','9','Unknown']
)

val_data = object_detector.DataLoader.from_pascal_voc(
    'images/validation',
    'images/validation',
    ['0', '1','2','3','4','5','6','7','8','9','Unknown']
)

## Step 4
### Auswahl der Architektur vom Model

 Auswahl zwischen EfficientDet-Lite[0-4]  
 [EfficientDet architecture](https://arxiv.org/abs/1911.09070)

Durchschnittliche Performance: Laufzeit Geschwindigkeit <> Durchschnittliche Genauigkeit.

| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|--------------------|-----------|---------------|----------------------|
| EfficientDet-Lite0 | 4.4       | 146           | 25.69%               |
| EfficientDet-Lite1 | 5.8       | 259           | 30.55%               |
| EfficientDet-Lite2 | 7.2       | 396           | 33.97%               |
| EfficientDet-Lite3 | 11.4      | 716           | 37.70%               |
| EfficientDet-Lite4 | 19.9      | 1886          | 41.96%               |

Durchschnitt wurde auf einem RasperryPi4 gemacht. Der Nvidia Jetson Nano Orin ist deutlich schneller.


In [ ]:
spec = model_spec.get('efficientdet_lite0')

##Step 5
### TensorFLow Model mit der training_data trainieren
wird einige Stunden beim vollen Model benötigen.
"Kleine Models" benötigen 1-2STD.

Beim "über trainieren" abbrechen durch Doppelklick auf den Run.



```

```



* Set `epochs = 20`, which means it will go through the training dataset 20 times. You can look at the validation accuracy during training and stop when you see validation loss (`val_loss`) stop decreasing to avoid overfitting.
* Set `batch_size = 4` here so you will see that it takes 15 steps to go through the 62 images in the training dataset.
* Set `train_whole_model=True` to fine-tune the whole model instead of just training the head layer to improve accuracy. The trade-off is that it may take longer to train the model.

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=4, train_whole_model=True, epochs=20, validation_data=val_data)

##Step 6
### Modelauswertung bei noch nicht zuvor gesehenen Bildern von der val_data (Step 6+7 Dauern länger nicht abbrechen).

(die Modelauswertung, sowie das Exportieren können eine ewigkeit Dauern ganz normal.
Auswertungs metrics --> [COCO](https://cocodataset.org/#detection-eval).

In [ ]:
model.evaluate(val_data)

##Step 7
### Exportieren vom TFLite Model

[full integer quantization](https://www.tensorflow.org/lite/performance/post_training_integer_quant). 

In [ ]:
model.export(export_dir='.', tflite_filename='detect.tflite')

##Step 8
### Auswertung vom TFLite Model
Exportiertes TFLite Model wird mit der Genauigkeit vom Originalen TFLite Model




```

```

Several factors can affect the model accuracy when exporting to TFLite:
* [Quantization](https://www.tensorflow.org/lite/performance/model_optimization) helps shrinking the model size by 4 times at the expense of some accuracy drop.
* The original TensorFlow model uses per-class [non-max supression (NMS)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH) for post-processing, while the TFLite model uses global NMS that's much faster but less accurate.
Keras outputs maximum 100 detections while tflite outputs maximum 25 detections.

 

In [ ]:
model.evaluate_tflite('detect.tflite', val_data)

###Download TFLite Model

In [ ]:
# Download the TFLite model to your local computer.
from google.colab import files
files.download('detect.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

________________________________________________________________________________

##Step 9 (Wird nur für ein Google Coral EdgeTPU TFlite Model benötigt, noch nicht mit Nvidia Jetson Nano Oring getestet)
### Compile the model for EdgeTPU

Zum Nutzen vom Coral EdgeTPU muss dieser+libary erst noch auf den Nvidia sbc installiert werden

[Google Coral EdgeTPU](https://coral.ai/).


In [ ]:
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
!sudo apt-get update
!sudo apt-get install edgetpu-compiler

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  59000      0 --:--:-- --:--:-- --:--:-- 59000
OK
deb https://packages.cloud.google.com/apt coral-edgetpu-stable main
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 https://packages.cloud.google.com/apt coral-edgetpu-stable InRelease [6,722 B]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,398 kB]
Ign:9 https://developer.download.nvidia.com/comput

Before compiling the `.tflite` file for the Edge TPU, it's important to consider whether your model will fit into the Edge TPU memory. 

The Edge TPU has approximately 8 MB of SRAM for [caching model paramaters](https://coral.ai/docs/edgetpu/compiler/#parameter-data-caching), so any model close to or over 8 MB will not fit onto the Edge TPU memory. That means the inference times are longer, because some model parameters must be fetched from the host system memory.

One way to elimiate the extra latency is to use [model pipelining](https://coral.ai/docs/edgetpu/pipeline/), which splits the model into segments that can run on separate Edge TPUs in series. This can significantly reduce the latency for big models.

The following table provides recommendations for the number of Edge TPUs to use with each EfficientDet-Lite model.

| Model architecture | Minimum TPUs | Recommended TPUs
|--------------------|-------|-------|
| EfficientDet-Lite0 | 1     | 1     |
| EfficientDet-Lite1 | 1     | 1     |
| EfficientDet-Lite2 | 1     | 2     |
| EfficientDet-Lite3 | 2     | 2     |
| EfficientDet-Lite4 | 2     | 3     |

If you need extra Edge TPUs for your model, then update `NUMBER_OF_TPUS` here:

In [ ]:
NUMBER_OF_TPUS = 5

!edgetpu_compiler detect.tflite --num_segments=$NUMBER_OF_TPUS

Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 3127 ms.

Input model: android.tflite
Input size: 4.22MiB
Output model: android_edgetpu.tflite
Output size: 5.57MiB
On-chip memory used for caching model parameters: 4.21MiB
On-chip memory remaining for caching model parameters: 3.29MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 267
Operation log: android_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider updating your model to use only operations supported by the Edge TPU. For details, visit g.co/coral/model-reqs.
Number of operations that will run on Edge TPU: 264
Number of operations that will run on CPU: 3
See the operation log file for individual operation details.
Compilation child proces

###Copy Metadata from tlfite model into the edgetpu Model and Download it

In [ ]:
populator_dst = metadata.MetadataPopulator.with_model_file('detect_edgetpu.tflite')

with open('detect.tflite', 'rb') as f:
  populator_dst.load_metadata_and_associated_files(f.read())

populator_dst.populate()
updated_model_buf = populator_dst.get_model_buffer()

In [ ]:
# Download the TFLite model compiled for EdgeTPU to your local computer.
from google.colab import files
files.download('detect_edgetpu.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>